# Discord Imagine



## Stable Diffusion


### Post An Attachment

#### the code

this is getting long.

update discord interaction with attachments?


In [ ]:
const {DEFAULT_APPLICATION} = importer.import('discord configuration')
const { safeurl } = importer.import('domain cache tools')
const mime = require('mime-types');
const {requestAuthQ} = importer.import('discord request')


async function patchInteractionImageAttachment(prompt, buff, token, content = 'image/png') {
  return await requestAuthQ({
    headers: {
      'Content-Type': 'multipart/form-data; boundary=boundary123'
    },
    method: 'PATCH',
    url: `webhooks/${DEFAULT_APPLICATION}/${token}/messages/@original`,
    body: `--boundary123
Content-Disposition: form-data; name="payload_json"
Content-Type: application/json

${JSON.stringify({
  content: 'Your ' + (content.includes('image') ? 'image' : content) + ' delivered: *' + prompt + '*',
  attachments: [{
    "id": 0,
    "description": prompt,
    "filename": safeurl(prompt) + '.' + mime.extension(content)
  }],
})}
--boundary123
Content-Disposition: form-data; name="files[0]"; filename="${safeurl(prompt)}.${mime.extension(content)}"
Content-Transfer-Encoding: base64
Content-Type: ${content}

${buff.toString('base64')}
--boundary123--`
  })
}


module.exports = {
  patchInteractionImageAttachment
}


### Proxy Request


#### the code


some sample interactions.

discord stability proxy?



In [ ]:
const llama = importer.import('create llm session')
const {patchInteractionImageAttachment} = importer.import('update discord interaction with attachments')
const doStableRequest = importer.import('stable diffusion request')

async function doHallucinate(interaction) {
  const {llmPrompt} = (await llama)
  let a1 = await llmPrompt('Imagine a scene that is much more exotic, weird, bizarre and detailed with this topic:\n' 
    + interaction.data.options[0].value + '\nFit as much detail as possible in to two or three sentences.')
  if(interaction.guild_id) {
    let q2 = 'Would this prompt be considered to be rated R or for mature audiences, answer Yes or No only?\n' + a1
    let a2 = await llmPrompt(q2)
    if(a2.includes('Yes')) {
      return await updateInteraction({
        content: 'I will not.'
      }, interaction.id, interaction.token)
    }
  }
  let buff = await doStableRequest(a1)
  return await patchInteractionImageAttachment(interaction.data.options[0].value + '* -> *' + a1, buff, interaction.token)
}


async function doImagineProxy(interaction) {
  if(interaction.guild_id) {
    const {llmPrompt} = (await llama)
    let q2 = 'Would this prompt be considered to be rated PG or safe for children, answer Yes or No only?\n' + interaction.data.options[0].value
    let a2 = await llmPrompt(q2)
    if(a2.includes('No')) {
      return await updateInteraction({
        content: 'I will not.'
      }, interaction.id, interaction.token)
    }
  }
  let buff = await doStableRequest(interaction.data.options[0].value)
  //const fs = require('fs')
  //fs.writeFileSync('/Users/briancullinan/Downloads/test.png', buff)
  //console.log(atob(result.data.images[0]))
  return await patchInteractionImageAttachment(interaction.data.options[0].value, buff, interaction.token)
}

module.exports = {
  doHallucinate,
  doImagineProxy,
  doStableRequest,
}


### Bot Commands


#### the code

discord mid journey commands?

export TOKENPATH=/Users/briancullinan/.credentials/discord-bot2.txt && export DEFAULT_APPLICATION=1335464182111535124 && npm run import -- "discord mid journey commands"


In [ ]:
const {
  registerCommand, getCommands, deleteCommand, updateCommand
} = importer.import('discord api')
const {interactionsCommands} = importer.import('discord gateway')
const {
  doHallucinate, doImagineProxy
} = importer.import('discord stability proxy')
const doVision = importer.import('discord ollama vision')
const {discordExpress, closeExpress} = importer.import('discord music player server')
const {doMention, doPrivate} = importer.import('discord llm interactions')

const ALL_COMMANDS = [
  'imagine',
  'hallucinate',
  'describe',
  'display',
]

// bot commands using new API, same names as above but lower-case
async function imagineCommands(guildId = null) {
  var cmd, cmdDef
  var commandResult = await getCommands(guildId)
  var commands = commandResult.map(command => command.name)

  interactionsCommands['promptMention'] = doMention
  interactionsCommands['promptPrivate'] = doPrivate

  cmdDef = {
    'name': 'imagine',
    'description': 'Prompt an image generator.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt stability diffusion with.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('imagine')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'imagine')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['imagine'] = doImagineProxy

  cmdDef = {
    'name': 'hallucinate',
    'description': 'Prompt an image generator with weirder content.',
    'options': [
      {
        'name': 'text',
        'description': 'Text to prompt the llm with then use with stable diffusion.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('hallucinate')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'hallucinate')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['hallucinate'] = doHallucinate


  cmdDef = {
    'name': 'describe',
    'description': 'Prompt Ollama 3.2 Vision with an image to describe.',
    'options': [
      {
        'name': 'url',
        'description': 'URL to the image you want a description of.',
        'required': true,
        'type': 3
      }
    ]
  }
  if(!commands.includes('describe')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'describe')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }
  interactionsCommands['describe'] = doVision

  
  interactionsCommands['startActivity'] = discordExpress
  interactionsCommands['endActivity'] = closeExpress
  
  cmdDef = {
    'name': 'display',
    'description': 'Launch an activity',
    'type': 4,
    'handler': 2,
  }
  if(!commands.includes('display')) {
    await registerCommand(cmdDef, null, guildId)
  } else {
    cmd = commandResult.filter(c => c.name == 'display')[0]
    if(cmdDef.name != cmd.name || cmdDef.description != cmd.description)
      await updateCommand(cmdDef, null, cmd.id, guildId)
  }

  
  var toRemove = commandResult.filter(c => !ALL_COMMANDS.includes(c.name))
  for(var i = 0; i < toRemove.length; i++) {
    await deleteCommand(toRemove[i].id, guildId)
  }

  return await getCommands()
}


module.exports = imagineCommands




### Ollama Vision



#### the code

discord ollama vision?


In [ ]:
const { request } = require('gaxios')
//const fs = require('fs')
const requestOllamaVision = importer.import('request ollama vision')
const {requestAuthQ} = importer.import('discord request')
const {URL} = require('url')

//const TEMP_DIR = os.tmpdir();

async function doVision(interaction) {
  // TODO: download the image file to temp
  let result
  if(interaction.data.options[0].value.includes('discord')
    && interaction.data.options[0].value.includes('/channels/')
  ) {
    // TODO: get the message reference instead
    let messageUrl = new URL(interaction.data.options[0].value)

    let channelId = (/\/channels\/(.*?)\/(.*?)\//gi).exec(interaction.data.options[0].value)[2]
    let messageId = path.basename(messageUrl.pathname)
    var message = await requestAuthQ({
      method: 'GET',
      url: `channels/${channelId}/messages/${messageId}`,
    })
    if(message.attachments && message.attachments[0]) {
      result = await request({
        url: message.attachments[0].url,
        method: 'GET',
      })
    } else if (message.message_snapshots && message.message_snapshots[0]
      && message.message_snapshots[0].message
      && message.message_snapshots[0].message.attachments
      && message.message_snapshots[0].message.attachments[0]
    ) {
      result = await request({
        url: message.message_snapshots[0].message.attachments[0].url,
        method: 'GET',
      })
    }
  } else {
    result = await request({
      url: interaction.data.options[0].value,
      method: 'GET',
    })
  }

  if(!result) {
    return await updateInteraction({
      content: 'Could not load: ' + interaction.data.options[0].value
    }, interaction.id, interaction.token)
  }

  // Describe the image using ollama vision proxy
  let content = await requestOllamaVision(Buffer.from(await result.data.arrayBuffer()))

  return await updateInteraction({
    content
  }, interaction.id, interaction.token)
}


module.exports = doVision
